In [ ]:

import os
from tinydb import TinyDB, table
import os
import sys
import datetime
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
from src.services.generateWeightedAverageTrade import generateWeightedAverageTrade


In [ ]:
def insertIntTableOfUpdates(updateTable, ticket, strategyName, strategyTimeFrame):

    def lambdaContains(
        x): return x['ticket'] == ticket['ticket'] and x['strategy'] == strategyName and x['timeFrame'] == strategyTimeFrame
    if updateTable.contains(lambdaContains):
        res = updateTable.search(lambdaContains)
        dbDate = datetime.datetime.strptime(res[0]['updateDate'],'%Y-%m-%d %H:%M:%S')
        ticketDate = datetime.datetime.strptime(ticket['updateDate'],'%Y-%m-%d %H:%M:%S')
        print("Ja existe uma entrada com os parametros informados.")
        if ticketDate < dbDate:
            print("A entrada do ticket é mais antiga que a entrada do banco de dados. Ela será atualizada.")
            updateTable.update(
                {"updateDate": ticket['updateDate']}, lambdaContains)
    else:
        print(
            "Nao existe uma sentrada com os parametros informados. Sera inserido uma nova.")
        updateTable.insert({"ticket": ticket['ticket'], "updateDate": ticket['updateDate'],
                           "strategy": strategyName, "timeFrame": strategyTimeFrame})


In [ ]:
path = os.path.join("..", 'DB', 'estrategys.json')
db = TinyDB(path)
estrategys = db.table('estrategys')
update = db.table("update")

In [ ]:
strategysForUpdate = []
total = len(estrategys)
print(total)
count = 0
for estrategy in estrategys:
    print(count," ", end="")
    result = estrategy['results']
    result['Weighted Average Trade'] = generateWeightedAverageTrade(result)
    estrategys.upsert(table.Document(
        {'Weighted Average Trade': result['Weighted Average Trade']}, doc_id=estrategy.doc_id))
    insertIntTableOfUpdates(update, estrategy, estrategy['strategy']['name'], estrategy['strategy']['timeFrame'])
    count += 1
    # break

